In [1]:
import numpy as np
import pandas as pd
import math
import re

# Better approaches
1. Can this be done by decision tree? e.g. sample split points +{+1,-1} around them

1. Sample points from rules, then do on their basis

1. Use classes


In [2]:
#FILE_NAME = "test.txt"
FILE_NAME = "input.txt"
DIAG=False
DIAG2 = False

In [3]:
eval("12<20000")


True

In [4]:
def buildRulesAndPartsFromLines(lines):
    
    cur_x = 0
    cur_y = 0
    
    map_df = pd.DataFrame(["#"])
    
    boundary = -1
    for i,line in enumerate(lines):
        if line[-1]=="\n": line = line[:-1]
        line = line.strip()        
        if len(line)==0: 
            boundary = i
    
    lines_rules = lines[:boundary]
    lines_gears = lines[boundary+1:]
    
    rules = buildRulesFromLines(lines_rules)
    gears,gears_outst_rules = buildGearsFromLines(lines_gears)
                    
    return rules,gears,gears_outst_rules
    

In [5]:
eval("True")

True

In [6]:
def evalConditionOnObject(rule,gear):
    if DIAG: print(rule)
    rule_cond,rule_outcome = rule
    if rule_cond == "True":
        return (True,rule_outcome)
    elif gear is None:
        return (False,None)
    else:
        #do the replacement
        if DIAG: print(rule_cond)
        mmm = re.match("^([a-zA-Z]+)",rule_cond)
        if DIAG: print("..",mmm[0],mmm[1])
        gear_var = mmm[1]
        
    if not gear_var is None:
        if DIAG:
            print(">>",gear)
            print(">>",gear_var)
            print(">>",gear[gear_var])
        rule_cond = rule_cond.replace(gear_var,str(gear[gear_var]),1)
    
    if eval(rule_cond):
        return (True,rule_outcome)
    else:
        return (False,None)        
        
    return None

evalConditionOnObject(('a>716', 'R'),{'x': '787', 'm': '2655', 'a': '1222', 's': '2876'})


(True, 'R')

In [7]:
def buildRulesFromLines(lines_rules):    
    rules = {}
    for line in lines_rules:
        z=line.find("{")
        name=line[:z]
        line=line[z+1:].strip()[:-1]
        
        
        res = []
        for x in line.split(","):            
            tmp = x.split(":")            
            if len(tmp)>1:
                res.append((tmp[0],tmp[1]))
            else:
                res.append(("True",tmp[0]))
        rules[name]=res
    return rules
               
        
y=["px{a<2006:qkq,m>2090:A,rfg}","pv{a>1716:R,A}","lnx{m>1548:A,A}"]
buildRulesFromLines(y)

{'px': [('a<2006', 'qkq'), ('m>2090', 'A'), ('True', 'rfg')],
 'pv': [('a>1716', 'R'), ('True', 'A')],
 'lnx': [('m>1548', 'A'), ('True', 'A')]}

In [8]:
def buildGearsFromLines(lines_gears):    
    gears = {}
    gears_vs_rules = {}
    for i,line in enumerate(lines_gears):
        line=line.strip()
        gear = {}
        for x in line[1:-1].split(","):
            feat,val = x.split("=")
            gear[feat]=val
            
            gears[i]=gear
            gears_vs_rules[i]=["in"]
    return gears,gears_vs_rules
               
        
y=["{x=787,m=2655,a=1222,s=2876}","{x=1679,m=44,a=2067,s=496}","{x=2036,m=264,a=79,s=2244}","{x=2461,m=1339,a=466,s=291}","{x=2127,m=1623,a=2188,s=1013}"]
buildGearsFromLines(y)

({0: {'x': '787', 'm': '2655', 'a': '1222', 's': '2876'},
  1: {'x': '1679', 'm': '44', 'a': '2067', 's': '496'},
  2: {'x': '2036', 'm': '264', 'a': '79', 's': '2244'},
  3: {'x': '2461', 'm': '1339', 'a': '466', 's': '291'},
  4: {'x': '2127', 'm': '1623', 'a': '2188', 's': '1013'}},
 {0: ['in'], 1: ['in'], 2: ['in'], 3: ['in'], 4: ['in']})

In [9]:
def processPart(part,rule_name,rules):
    """
    part_outst_rules = [(),(),()]
    """
    part_outst_rules = rules[rule_name]
    while len(part_outst_rules) > 0:
        rule = part_outst_rules[0]
        part_outst_rules = part_outst_rules[1:]
        
        if DIAG: print("processPart -- part =",part)
        switch_over,new_rule = evalConditionOnObject(rule,part)
        if new_rule == "R":
            return False
        elif new_rule == "A":
            return True
        elif not switch_over:
            pass
        else:
            # switch over
            return processPart(part,new_rule,rules)
    
    print("Didn't reach conculsion")
    return False     
    
        

In [10]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    rules,parts,parts_outst_rules = buildRulesAndPartsFromLines(lines)
   
    
    
    return rules,parts,parts_outst_rules
    

In [11]:
def score(part):
    res = 0
    for k,v in part.items():
        res+=int(v)
    return res

In [12]:
def startingGearRange():
    part = {"x":(1,4000),"m":(1,4000),"a":(1,4000),"s":(1,4000)}
    return part

In [13]:
def splitGearRange(inGearRng,returnLeft,split_on,split_val,include_val_in_left):
    x,y = inGearRng[split_on]
        
    if (include_val_in_left):
        z = split_val
    else:
        z = split_val-1
    
    if (returnLeft and z < x) or (not returnLeft and z+1>y):
        return None
    elif returnLeft and z>=x:
        newRng = inGearRng.copy()
        newRng[split_on]=(x,z)
        return newRng
    else:
        newRng = inGearRng.copy()
        newRng[split_on]=(z+1,y)
        return newRng
    

In [14]:
def parseCondition(str_condtion):
    mmm = re.match("([a-zA-Z]+)([<>])(\d+$)",str_condtion)    
    if DIAG: print("-->",str_condtion,"\n",mmm)
    var = mmm[1]
    oper=mmm[2]
    include_in_left = (oper == ">")
    num=int(mmm[3])
    return var,num,include_in_left

parseCondition("a>1716")

('a', 1716, True)

In [15]:
def getSamplePart(objRng):
    sampleObj = {}
    for key,val in objRng.items():
        sampleObj[key]=val[0]
    return sampleObj

print(startingGearRange())
print(getSamplePart(splitGearRange(startingGearRange(),False,"m",1987,True)))

{'x': (1, 4000), 'm': (1, 4000), 'a': (1, 4000), 's': (1, 4000)}
{'x': 1, 'm': 1988, 'a': 1, 's': 1}


In [16]:
def evaluateRangeForCondition(objRng,cond):
    objPart = getSamplePart(objRng)
    return evalConditionOnObject(cond,objPart)    

In [17]:
def evaluateRangeForRule(gearRange,rule,rules):
    return evaluateRangeForMultipleConditions(gearRange,rules[rule])    

def evaluateRangeForMultipleConditions(gearRange,conditions):
    # conditions are a list    
        
    res_next_evals = []
        
    
    cond = conditions[0]
    if evalConditionOnObject(cond,None)[0]: # outright true
        res_next_evals = [(gearRange,cond[1])]
    else:
        # something to fcheck
        if DIAG: print("~~",conditions)
        split_on,split_val,include_val_in_left=parseCondition(cond[0])
    
        newRngLeft=splitGearRange(gearRange,returnLeft=True,split_on=split_on, split_val=split_val, 
                                  include_val_in_left=include_val_in_left)
        newRngRight=splitGearRange(gearRange,returnLeft=False,split_on=split_on, split_val=split_val,
                                   include_val_in_left=include_val_in_left)
            
        for newRng in [newRngLeft,newRngRight]:
            # if two ranges are not None then one is True and the other one is False
            if newRng is not None:
                resBool, resCons = evaluateRangeForCondition(newRng,cond)
            
                if resBool:
                    # pop this newRng on queue to process (newRng,resCons)
                    res_next_evals += [(newRng,resCons)]
                else:
                    res_tmp = evaluateRangeForMultipleConditions(newRng, conditions[1:])
                    res_next_evals += res_tmp
                    
                
    return res_next_evals

In [18]:
def singleRangeCardinality(objRng):
    res = 1
    for x,val in objRng.items():
        res *= (val[1]-val[0]+1)
    return res

def tallyUpRanges(rangeOutcomes):
    res = 0
    for objRng,outcome in rangeOutcomes:
        if outcome == "A":
            res += singleRangeCardinality(objRng)
    return res
    

In [19]:
singleRangeCardinality({'x': (1, 4000), 'm': (2091, 4000), 'a': (2006, 4000), 's': (1, 1350)})

20576430000000

In [20]:
rules,_,_=parseFile(FILE_NAME)
if DIAG:
    print(rules)

startRange=startingGearRange()

to_eval = [(startRange,"in")]
decided = []
counter = 0
while len(to_eval)>0 and counter < 1000000:
    counter += 1
    rngObj,ruleName =to_eval[0]
    to_eval = to_eval[1:]
    if ruleName in ["A","R"]:
        decided += [(rngObj,ruleName)]
    else:
        to_eval += evaluateRangeForRule(rngObj,ruleName,rules)
    
if len(to_eval) > 0: print("This has gone south, {} elements left!!!!".format(len(to_eval)))
if DIAG2: print(decided)
res = tallyUpRanges(decided)
print("Result: ",res)

Result:  130745440937650
